In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV

In [31]:
df=pd.read_csv('/kaggle/input/loan-pred-traincsv/Loan pred_train.csv')
test= pd.read_csv("/kaggle/input/loan-pred-traincsv/Loan Pred_test.csv")

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         600 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [33]:
df['LoanAmount'] =df['LoanAmount'] * 1000
test['LoanAmount'] =test['LoanAmount'] * 1000

In [34]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128000.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66000.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120000.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141000.0,360.0,1.0,Urban,Y


In [35]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146412.162162,342.00000,0.842199
std,6109.041673,2926.248369,85587.325236,65.12041,0.364878
min,150.000000,0.000000,9000.000000,12.00000,0.000000
25%,2877.500000,0.000000,100000.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128000.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168000.000000,360.00000,1.000000
max,81000.000000,41667.000000,700000.000000,480.00000,1.000000


In [36]:
test.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,367.000000,367.000000,362.000000,361.000000,338.000000
mean,4805.599455,1569.577657,136132.596685,342.537396,0.825444
std,4910.685399,2334.232099,61366.652393,65.156643,0.380150
min,0.000000,0.000000,28000.000000,6.000000,0.000000
25%,2864.000000,0.000000,100250.000000,360.000000,1.000000
50%,3786.000000,1025.000000,125000.000000,360.000000,1.000000
75%,5060.000000,2430.500000,158000.000000,360.000000,1.000000
max,72529.000000,24000.000000,550000.000000,480.000000,1.000000


In [37]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           14
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [38]:
df['Gender'].fillna((df['Gender'].mode()[0]),inplace=True)
df['Married'].fillna((df['Married'].mode()[0]),inplace=True)
df['Dependents'].fillna((df['Dependents'].mode()[0]),inplace=True)
df['Self_Employed'].fillna((df['Self_Employed'].mode()[0]),inplace=True)

test['Gender'].fillna((df['Gender'].mode()[0]),inplace=True)
test['Married'].fillna((df['Married'].mode()[0]),inplace=True)
test['Dependents'].fillna((df['Dependents'].mode()[0]),inplace=True)
test['Self_Employed'].fillna((df['Self_Employed'].mode()[0]),inplace=True)

In [39]:
df['Dependents'].replace('+','')

0       0
1       1
2       0
3       0
4       0
       ..
609     0
610    3+
611     1
612     2
613     0
Name: Dependents, Length: 614, dtype: object

In [40]:
df['Dependents']=df['Dependents'].apply(lambda x:str(x).replace('+','')if '+' in str(x) else str(x))
df['Dependents']=df['Dependents'].apply(lambda x:int(x))

test['Dependents']=test['Dependents'].apply(lambda x:str(x).replace('+','')if '+' in str(x) else str(x))
test['Dependents']=test['Dependents'].apply(lambda x:int(x))

In [41]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [42]:
df['LoanAmount'].fillna((df['LoanAmount'].median()),inplace=True)
df['Loan_Amount_Term'].fillna((df['Loan_Amount_Term'].median()),inplace=True)
df['Credit_History'].fillna((df['Credit_History'].median()),inplace=True)

test['LoanAmount'].fillna((test['LoanAmount'].median()),inplace=True)
test['Loan_Amount_Term'].fillna((test['Loan_Amount_Term'].median()),inplace=True)
test['Credit_History'].fillna((test['Credit_History'].median()),inplace=True)


In [43]:
from sklearn.preprocessing import LabelEncoder

lb=LabelEncoder()
df['Loan_Status']=lb.fit_transform(df['Loan_Status'])

In [44]:
df.drop("Loan_ID", inplace=True,axis=1)


In [45]:
test.drop("Loan_ID", inplace=True,axis=1)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             614 non-null    object 
 1   Married            614 non-null    object 
 2   Dependents         614 non-null    int64  
 3   Education          614 non-null    object 
 4   Self_Employed      614 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         614 non-null    float64
 8   Loan_Amount_Term   614 non-null    float64
 9   Credit_History     614 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 57.7+ KB


In [47]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [48]:
cat = ['Gender', 'Married','Education', 'Self_Employed','Property_Area']

for i in cat:
    df[i]=lb.fit_transform(df[i])
    test[i]=lb.fit_transform(test[i])
    

In [49]:
df['Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
test['Income'] = test['ApplicantIncome'] + test['CoapplicantIncome']

df['Installment'] =df['LoanAmount']/df['Loan_Amount_Term']
test['Installment'] =test['LoanAmount']/test['Loan_Amount_Term']

df['Ratio'] =df['Income']/df['LoanAmount']
test['Ratio'] =test['Income']/test['LoanAmount']



In [50]:
X=df.drop("Loan_Status",axis=1)
y=df['Loan_Status']

In [51]:
from sklearn.model_selection import train_test_split as split

X_train, X_test, y_train, y_test = split(X,y, test_size=0.15, random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((521, 14), (93, 14), (521,), (93,))

In [52]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test=scaler.transform(test)

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(max_iter = 100)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy : {}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy : 0.8279569892473119


In [54]:
from sklearn import svm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

#Create a svm Classifier
sv = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
sv.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = sv.predict(X_test)
print("Accuracy SVM:",accuracy_score(y_test, y_pred))
print("Precision SVM:",precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall SVM:",recall_score(y_test, y_pred))

Accuracy SVM: 0.8279569892473119
Precision SVM: 0.8170731707317073
Recall SVM: 0.9852941176470589


In [55]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)



In [56]:
from sklearn.model_selection import KFold,LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
#clf = svm.SVC(kernel='linear')
cv = KFold(n_splits=6)
loo = LeaveOneOut()

predictions =  cross_val_score(logreg, X, y, cv=cv)
predictions.mean()

0.809537407195888

In [57]:
from xgboost import XGBClassifier

clf= XGBClassifier(learning_rate=0.08, n_estimators=153, max_depth=1,
                        min_child_weight=4, 
                         seed=27)

clf.fit(X_train, y_train)

# y_pred = logreg.predict(X_test)
y_pred = clf.predict(X_test)
print('XGboost accuracy score {}'. format(accuracy_score(y_test, y_pred)))

XGboost accuracy score 0.8172043010752689


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.34)

In [102]:
from xgboost import XGBClassifier

clf= XGBClassifier(learning_rate=0.08, n_estimators=153, max_depth=1,
                        min_child_weight=4, 
                         seed=27)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
XG_fit_time = scores['fit_time'].mean()
XG_score_time = scores['score_time'].mean()
XG_accuracy = scores['test_accuracy'].mean()
XG_precision = scores['test_precision_macro'].mean()
XG_recall = scores['test_recall_macro'].mean()
XG_f1 = scores['test_f1_weighted'].mean()
XG_roc = scores['test_roc_auc'].mean()

In [59]:
LR = LogisticRegression()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LR, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
LR_fit_time = scores['fit_time'].mean()
LR_score_time = scores['score_time'].mean()
LR_accuracy = scores['test_accuracy'].mean()
LR_precision = scores['test_precision_macro'].mean()
LR_recall = scores['test_recall_macro'].mean()
LR_f1 = scores['test_f1_weighted'].mean()
LR_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
decision_tree = DecisionTreeClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(decision_tree, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
dtree_fit_time = scores['fit_time'].mean()
dtree_score_time = scores['score_time'].mean()
dtree_accuracy = scores['test_accuracy'].mean()
dtree_precision = scores['test_precision_macro'].mean()
dtree_recall = scores['test_recall_macro'].mean()
dtree_f1 = scores['test_f1_weighted'].mean()
dtree_roc = scores['test_roc_auc'].mean()

In [61]:
SVM = SVC(probability = True)

scoring = ['accuracy','precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(SVM, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
SVM_fit_time = scores['fit_time'].mean()
SVM_score_time = scores['score_time'].mean()
SVM_accuracy = scores['test_accuracy'].mean()
SVM_precision = scores['test_precision_macro'].mean()
SVM_recall = scores['test_recall_macro'].mean()
SVM_f1 = scores['test_f1_weighted'].mean()
SVM_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
LDA = LinearDiscriminantAnalysis()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LDA, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
LDA_fit_time = scores['fit_time'].mean()
LDA_score_time = scores['score_time'].mean()
LDA_accuracy = scores['test_accuracy'].mean()
LDA_precision = scores['test_precision_macro'].mean()
LDA_recall = scores['test_recall_macro'].mean()
LDA_f1 = scores['test_f1_weighted'].mean()
LDA_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
QDA = QuadraticDiscriminantAnalysis()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(QDA, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
QDA_fit_time = scores['fit_time'].mean()
QDA_score_time = scores['score_time'].mean()
QDA_accuracy = scores['test_accuracy'].mean()
QDA_precision = scores['test_precision_macro'].mean()
QDA_recall = scores['test_recall_macro'].mean()
QDA_f1 = scores['test_f1_weighted'].mean()
QDA_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminan

In [64]:
random_forest = RandomForestClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(random_forest, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
forest_fit_time = scores['fit_time'].mean()
forest_score_time = scores['score_time'].mean()
forest_accuracy = scores['test_accuracy'].mean()
forest_precision = scores['test_precision_macro'].mean()
forest_recall = scores['test_recall_macro'].mean()
forest_f1 = scores['test_f1_weighted'].mean()
forest_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
KNN = KNeighborsClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(KNN, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
KNN_fit_time = scores['fit_time'].mean()
KNN_score_time = scores['score_time'].mean()
KNN_accuracy = scores['test_accuracy'].mean()
KNN_precision = scores['test_precision_macro'].mean()
KNN_recall = scores['test_recall_macro'].mean()
KNN_f1 = scores['test_f1_weighted'].mean()
KNN_roc = scores['test_roc_auc'].mean()

In [66]:
bayes = GaussianNB()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(bayes, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
bayes_fit_time = scores['fit_time'].mean()
bayes_score_time = scores['score_time'].mean()
bayes_accuracy = scores['test_accuracy'].mean()
bayes_precision = scores['test_precision_macro'].mean()
bayes_recall = scores['test_recall_macro'].mean()
bayes_f1 = scores['test_f1_weighted'].mean()
bayes_roc = scores['test_roc_auc'].mean()

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
models_initial = pd.DataFrame({
    'Model'       : ['XG','Logistic Regression', 'Decision Tree', 'Support Vector Machine', 'Linear Discriminant Analysis', 'Quadratic Discriminant Analysis', 'Random Forest', 'K-Nearest Neighbors', 'Bayes'],
    'Fitting time': [XG_fit_time,LR_fit_time, dtree_fit_time, SVM_fit_time, LDA_fit_time, QDA_fit_time, forest_fit_time, KNN_fit_time, bayes_fit_time],
    'Scoring time': [XG_score_time,LR_score_time, dtree_score_time, SVM_score_time, LDA_score_time, QDA_score_time, forest_score_time, KNN_score_time, bayes_score_time],
    'Accuracy'    : [XG_accuracy,LR_accuracy, dtree_accuracy, SVM_accuracy, LDA_accuracy, QDA_accuracy, forest_accuracy, KNN_accuracy, bayes_accuracy],
    'Precision'   : [XG_precision,LR_precision, dtree_precision, SVM_precision, LDA_precision, QDA_precision, forest_precision, KNN_precision, bayes_precision],
    'Recall'      : [XG_recall,LR_recall, dtree_recall, SVM_recall, LDA_recall, QDA_recall, forest_recall, KNN_recall, bayes_recall],
    'F1_score'    : [XG_f1,LR_f1, dtree_f1, SVM_f1, LDA_f1, QDA_f1, forest_f1, KNN_f1, bayes_f1],
    'AUC_ROC'     : [XG_roc,LR_roc, dtree_roc, SVM_roc, LDA_roc, QDA_roc, forest_roc, KNN_roc, bayes_roc],
    }, columns = ['Model', 'Fitting time', 'Scoring time', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'AUC_ROC'])

models_initial.sort_values(by='Accuracy', ascending=False)

,Model,Fitting time,Scoring time,Accuracy,Precision,Recall,F1_score,AUC_ROC
1,Logistic Regression,0.009849,0.005647,0.817143,0.833854,0.697024,0.785541,0.689341
0,XG,0.025144,0.007386,0.816905,0.842504,0.706786,0.791866,0.738151
4,Linear Discriminant Analysis,0.006568,0.005806,0.809405,0.817379,0.695833,0.772883,0.738435
6,Random Forest,0.252229,0.025076,0.804048,0.789914,0.711905,0.778160,0.762543
3,Support Vector Machine,0.030776,0.006371,0.801905,0.801356,0.683333,0.762164,0.696684
8,Bayes,0.001294,0.005487,0.794524,0.781415,0.687500,0.761844,0.725340
7,K-Nearest Neighbors,0.001591,0.007510,0.757143,0.736689,0.655952,0.731741,0.701743
2,Decision Tree,0.003874,0.005776,0.718571,0.670335,0.663690,0.712517,0.663690
5,Quadratic Discriminant Analysis,0.002600,0.007121,0.687857,0.656781,0.648810,0.668451,0.718027


In [119]:
models = [LogisticRegression(),
          XGBClassifier(learning_rate=0.08, n_estimators=153, max_depth=1,
                        min_child_weight=4, 
                         seed=27),
         DecisionTreeClassifier(),
         SVC(probability = True),
         LinearDiscriminantAnalysis(),
         QuadraticDiscriminantAnalysis(),
         RandomForestClassifier(),
         KNeighborsClassifier(),
         GaussianNB()]

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10)

In [175]:
for model in models:
    scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=20)

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/lib/python3.7/site-packages/sklearn/discriminan

In [176]:
models_ens = list(zip(['XG','LR', 'DT', 'SVM', 'LDA', 'QDA', 'RF', 'KNN', 'NB'], models))

model_ens = VotingClassifier(estimators = models_ens, voting = 'hard')
model_ens.fit(X_train, y_train)
pred = model_ens.predict(X_test)
#prob = model_ens.predict_proba(X_test)[:,1]

acc_hard = accuracy_score(y_test, pred)
prec_hard = precision_score(y_test, pred)
recall_hard = recall_score(y_test, pred)
f1_hard = f1_score(y_test, pred)
roc_auc_hard = 'not applicable'

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [177]:
model_ens = VotingClassifier(estimators = models_ens, voting = 'soft')
model_ens.fit(X_train, y_train)
pred = model_ens.predict(X_test)
prob = model_ens.predict_proba(X_test)[:,1]

acc_soft = accuracy_score(y_test, pred)
prec_soft = precision_score(y_test, pred)
recall_soft = recall_score(y_test, pred)
f1_soft = f1_score(y_test, pred)
roc_auc_soft = roc_auc_score(y_test, prob)

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [178]:
models_ensembling = pd.DataFrame({
    'Model'       : ['Ensebling_hard', 'Ensembling_soft'],
    'Accuracy'    : [acc_hard, acc_soft],
    'Precision'   : [prec_hard, prec_soft],
    'Recall'      : [recall_hard, recall_soft],
    'F1_score'    : [f1_hard, f1_soft],
    'AUC_ROC'     : [roc_auc_hard, roc_auc_soft],
    }, columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'AUC_ROC'])

models_ensembling.sort_values(by='Accuracy', ascending=False)

,Model,Accuracy,Precision,Recall,F1_score,AUC_ROC
0,Ensebling_hard,0.798450,0.784483,0.989130,0.875000,not applicable
1,Ensembling_soft,0.782946,0.780702,0.967391,0.864078,0.670682


In [180]:
df['Loan_Status'].value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

In [182]:
result = model_ens.predict(test)
test_id = pd.read_csv("/kaggle/input/loan-pred-traincsv/Loan Pred_test.csv")
test_id = test_id.Loan_ID


In [183]:
result = pd.DataFrame(result)
test_id = pd.DataFrame(test_id)

In [184]:
submission = pd.merge(test_id,result,left_index=True,right_index=True)

In [185]:
submission.rename(columns={0:'Loan_Status'},inplace=True)

In [186]:
submission['Loan_Status'].replace({0:'N'},inplace=True)
submission['Loan_Status'].replace({1:'Y'},inplace=True)

In [187]:
submission.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y


In [188]:
submission.to_csv('Loan.csv',index=False)

In [189]:
submission['Loan_Status'].value_counts()

Y    305
N     62
Name: Loan_Status, dtype: int64